In [1]:
!ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_0/*nonpeaks_negatives.bed | wc -l
#1354


1354


In [2]:
# !ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_0/*nonpeaks_negatives.bed | head

In [3]:
!ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_1/*nonpeaks_negatives.bed | wc -l
#1354


1354


In [4]:
!ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_2/*nonpeaks_negatives.bed | wc -l
#1354


1354


In [5]:
!ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_3/*nonpeaks_negatives.bed | wc -l
#1354


1354


In [6]:
!ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_4/*nonpeaks_negatives.bed | wc -l
#1354


1354


In [1]:
!squeue --user=$USER | wc -l

# 216

130


In [2]:
# !rm errors.log

In [3]:
# !grep -i "error" ./local_logs/slurm.step5NegativesNoPeaksBackground.combined.err | tr '\n' ' ' >> errors.log



In [4]:
# !grep -E "error|failed|exception" ./local_logs/slurm.step5NegativesNoPeaksBackground.combined.err | tr '\n' ' ' >> errors1.log
# !grep -E -o ".{0,100}(error|failed|exception).{0,100}" ./local_logs/slurm.step5NegativesNoPeaksBackground.combined.err >> errors1.log
# !grep -i -E -o ".{0,100}(error|failed|exception).{0,100}" ./local_logs/slurm.step5NegativesNoPeaksBackground.combined.err >> errors1.log


In [5]:
# head ./loca

In [6]:
!squeue --user=$USER
# | tail

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
      54311036_288  akundaje step62.b     eila  R    9:43:07      1 sh02-12n09
       54311036_12  akundaje step62.b     eila  R   20:01:03      1 sh02-12n09
       54311036_13  akundaje step62.b     eila  R   20:01:03      1 sh02-12n09
        54311036_5  akundaje step62.b     eila  R   20:09:04      1 sh02-12n10
       54311036_25  akundaje step62.b     eila  R   19:16:00      1 sh02-12n08
       54311036_26  akundaje step62.b     eila  R   19:16:00      1 sh02-12n08
       54311036_27  akundaje step62.b     eila  R   19:16:00      1 sh02-12n08
       54311036_28  akundaje step62.b     eila  R   19:16:00      1 sh02-12n08
      54311036_346  akundaje step62.b     eila  R    4:27:26      1 sh03-16n09
      54311036_287  akundaje step62.b     eila  R    9:43:07      1 sh02-12n11
      54311036_348  akundaje step62.b     eila  R    4:07:16      1 sh03-16n09
      54311036_347  akundaje step62.b     eila

In [7]:
# !scancel 54297785
# !scancel 54291987

In [8]:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/*/*/fold_0/step62.bpnetPipeline/evaluation/overall_report.html | wc -l 
#117


162


In [32]:
# !scancel 54084823

In [51]:
# !squeue -u $USER -o "%i %j" | grep step5Neg


In [44]:
 # scp -r eila@login.sherlock.stanford.edu:/scratch/users/eila/encode_pseudobulks_model_training/human/ENCSR037JDN/ENCFF933KCP/fold_1_30000_20240912_182023/evaluation . 

# !rm ./local_logs/slurm_samools_err.combined.err
# !rm ./local_logs/slurm_samools_out.combined.out